First part of the code:

- Get data of toronto neighborhoods from Wikipedia
- Get toronto borough/neighborhood coordinates from csv file
- Call Foursquare to get venues and locations
- Save venue data in excel file

In [3]:
import requests
import pandas as pd
import numpy as np
import xlrd
import xlsxwriter
from bs4 import BeautifulSoup

In [ ]:
# Define constants for Foursquare URL
CLIENT_ID = 'STUXIOFXH55TITYVUTGKIDG3CAG50XD0B3SM2XE2WWB3IU1Q'
CLIENT_SECRET = 'AU3QRD5UCY5MRYBX3K2E3MMBOGCVNTA2WMHPZTDEN4IJ4CLZ'
VERSION = '20180605'
Limit = 100

In [1]:
# Function to call Foursquare App and assign found venues to Neighborhoods and boroughs
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # URL of respective Foursquare App
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            Limit)

        # Results from Foursquare
        # Note: Due to maximum amount of requests per day, errors might occur
        results = requests.get(url).json()['response']['groups'][0]['items']


        # Values to return
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)

In [ ]:
# Get Toronto location data from Wikipedia:

wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(wiki_url,'lxml')
print(soup.prettify())

wiki_table = soup.find_all('table',{'class':'wikitable sortable'})
table = wiki_table[0]

header_row = table.find('tr')
col_names = []


content = []
temp_post =[]
temp_boro = []
temp_neighbor = []

# Iterate through wikipedia HTML text and get values
for tr in table.find_all('tr'):

    temp_list =[]

    td_all = tr.find_all('td')

    if not td_all:
        continue

    post, boro, neighbor = [td.text.strip() for td in td_all[0:3]]

    temp_list.append(post)
    temp_list.append(boro)
    temp_list.append(neighbor)

    content.append(temp_list)


# Assign columns names to new df
df_toronto = pd.DataFrame(data=content, columns=['PostalCode', 'Borough', 'Neighborhood'])

df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']

In [ ]:
# Save df as "toronto_original.xlsx"
writer = pd.ExcelWriter(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_original', engine='xlsxwriter')

df_toronto.to_excel(writer)

writer.save()

In [ ]:
# Read df with coordinates and merge with toronto postal data
df_geo = pd.read_csv(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\Geospatial_Toronto.csv')

df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

df_torontoGEO = pd.merge(df_toronto, df_geo,how='left', on='PostalCode')

# Toronto venues from Foursquare
toronto_venues = getNearbyVenues(names=df_torontoGEO['Neighborhood'], latitudes=df_torontoGEO['Latitude'], longitudes=df_torontoGEO['Longitude'])

In [ ]:
# Save nw df with venues as "toronto_venues.xlsx"
toronto_excel = pd.ExcelWriter(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_venues.xlsx', engine='xlsxwriter')

toronto_venues.to_excel(toronto_excel, sheet_name='Toronto Venues')

toronto_excel.save()

Second part of code:

- Create different data frames from toronto_venues
- Group the data frames to explore data
- Save each table into the same excel file
- Additional: Check values in MS Excel

In [ ]:
import pandas as pd
import numpy as np
import xlrd

In [ ]:
# Read the excel file with Toronto data from other python file:
toronto_venues = pd.read_excel(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_venues.xlsx')

toronto_venues.head()

In [ ]:
# Drop location data
venue_check = toronto_venues.drop(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude'], axis=1)

# Create different data frames based on grouping
to_venue_by_boro = venue_check.groupby('Neighborhood', as_index=False)['Venue','Venue Category'].agg(lambda x: len(x.value_counts()))
to_venue_by_boro.sort_values(by='Neighborhood')


to_venue_by_cat = venue_check.groupby('Venue Category', as_index=False)['Neighborhood'].agg(lambda x: len(x.value_counts()))
to_venue_by_cat.sort_values(by='Venue Category')


to_venue_by_venue = venue_check.groupby('Venue', as_index=False)['Neighborhood', 'Venue Category'].agg(lambda x: len(x.value_counts()))
to_venue_by_venue.sort_values(by='Venue')

In [ ]:
to_groups_excel = pd.ExcelWriter(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_groups.xlsx', engine='xlsxwriter')

# Export the grouped DataFrames to new excel file for later use or individual analysis in MS Excel.
to_venue_by_boro.to_excel(to_groups_excel, sheet_name='Neighborhood boroughs Overview')
to_venue_by_cat.to_excel(to_groups_excel, sheet_name='Venue Categories Overview')
to_venue_by_venue.to_excel(to_groups_excel, sheet_name='Venues Overview')

to_groups_excel.save()

Third part of code:

- Explore max values of data frames in Python
- Load data from geojson file
- Create choropleth chart to see the distribution of venues in Toronto


In [ ]:
import pandas as pd
import numpy as np
import xlrd
import folium
import json


In [ ]:
path_to_groups = r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_groups.xlsx'

# Data to create geographical charts
to_geojson = r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_crs84.geojson'
to_lat = 43.653908
to_long = -79.384293


In [ ]:
# Read Excel data into dataframes again
# Note: If data was sorted or changed in MS Excel, pandas might return wrong results due to mixed indices
# Therefore, each index has been removed from excel files and is re-apllied when read into pandas
to_venue_by_boro = pd.read_excel(path_to_groups, sheet_name='Neighborhood boroughs Overview')
to_venue_by_cat = pd.read_excel(path_to_groups, sheet_name='Venue Categories Overview')
to_venue_by_venue = pd.read_excel(path_to_groups, sheet_name='Venues Overview')

In [ ]:
# Borough with most categories:
i_max = to_venue_by_boro['Venue Category'].idxmax()
print('The neighborhood with most different categories: ',to_venue_by_boro['Neighborhood'].iloc[i_max])

# The most common venue category in boroughs:
i_max = to_venue_by_cat['Neighborhood'].idxmax()
print('The most common category: ',to_venue_by_cat['Venue Category'].iloc[i_max])

# The most common venue name:
i_max = to_venue_by_venue['Neighborhood'].idxmax()
print('The most common venue name: ', to_venue_by_venue['Venue'].iloc[i_max])

In [3]:
# Back to the main data frame:

# Create choropleth chart for some Toronto areas.
# Note: Geojson file and key_on feature of map prone to errors
to_geojson = r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto2.geojson'

toronto_data = pd.read_excel(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_venues.xlsx')

In [ ]:
# Load data from geojson found on internet
geo = read_json = json.load(open(to_geojson))

df_to_choro = toronto_data.groupby(['Neighborhood'], as_index=False)['Venue'].agg(lambda x: len(x.value_counts()))

# Create map from folium with threshold
to_map = folium.Map(location=[to_lat, to_long], zoom_start=10, tiles='Mapbox bright')

threshold_scale_1 = np.linspace(df_to_choro['Venue'].min(),
                              df_to_choro['Venue'].max(),
                              6, dtype=int)


In [ ]:
# Create choropleth map
# Note: Due to differences in data from Foursquare and geojson, choropleth map might be prone to errors
to_map.choropleth(
    geo_data=geo,
    name='to_choropleth',
    data=df_to_choro,
    columns=['Neighborhood', 'Venue'],
    key_on='feature.properties.name',
    fill_color='BuGn',
    fill_opacity=0.7,
    line_opacity=0.3,
    threshold_scale=threshold_scale_1)

to_map

Fourth part of code:

- Create data frame that only contains information on restaurants of food stores in Toronto
- Use kmeans cluster to create four clusters for the Neighborhoods 
	based on their restaurant density
- Save the grouping into a new excel file


In [ ]:
import pandas as pd
import numpy as np
import xlrd
import xlsxwriter
import sklearn
import regex
from sklearn.cluster import KMeans
import folium

In [ ]:
toronto_venues = pd.read_excel(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\toronto_venues.xlsx')

# List of food related places excluding shops but coffee shops:
list_food = ['Restaurant', 'Cafe', 'Coffee', 'Food', 'Bar', 'Pub', 'Bakery', 'Beer', 'Ice', 'Burger', 'Diner', 'Pizza', 'Steak', 'Sushi', 'Sandwich', 'Mexican', 'Italian', 'Breakfast']

df_food = toronto_venues

# Iterate through df_food and drop all venue categories that have nothing to do with food
for i, row in df_food.iterrows():

    if not regex.search(r'\L<list_food>', row['Venue Category'], list_food=list_food):
        df_food.drop([i], inplace=True)

In [ ]:
# Prepare data frames for clustering. See which neighborhoods fall into different clusters based on
# their density of restaurants, bars, cafes etc.
num_cluster = 4

df_food_grouped = df_food.groupby('Neighborhood', as_index=False)['Venue Category'].count()

toronto_food_cluster = df_food_grouped.drop('Neighborhood', axis=1)

In [ ]:
# Initiate clustering with k means
kmean_cluster = KMeans(n_clusters=num_cluster, random_state=0).fit(toronto_food_cluster)

df_cluster = toronto_food_cluster

df_cluster['Neighborhood'] = df_food_grouped['Neighborhood']

df_cluster['Cluster'] = kmean_cluster.labels_

df_cluster = df_cluster[['Neighborhood', 'Venue Category', 'Cluster']]

print(df_cluster.head())

In [ ]:
#Save data frame with clusters as "food cluster neighbor.xlsx"
# Note: Other statistical data for each cluster can also be quickly evaluated in MS Excel
excel_cluster = pd.ExcelWriter(r'C:\Users\User01\Documents\Online Courses\IBM Data Science\Course  9 - Capstone\Main Project\food cluster neighbor.xlsx', engine='xlsxwriter')

df_cluster.to_excel(excel_cluster, sheet_name='Venue Category Clusters')

excel_cluster.save()